```

```
Δ=Power[r,2]-2r+Power[a,2]
ρ=Sqrt[Power[r,2]+Power[a,2]+Power[Cos[θ],2]]
P=Power[r,2]+Power[a,2]-ab
R=Power[P,2]-Δ[Power[\(40)b-a\(41),2]+q]
Θ=q-Power[Cos[θ],2](Divide[Power[b,2],Power[Sin[θ],2]]-Power[a,2])


For dp_theta: 
A = r^2+a^2-ab = R 
B = (r^2-2r+a^2)(q-Cos[θ]^2(b^2/Sin[θ]^2-a^2)) = ΔΘ
C = 2(Power[r,2]-2r+Power[a,2])(Power[r,2]+Power[a,2]+Power[Cos[θ],2]) = 2ΔPower[ρ,2]
```


```

In [111]:

#[derive(Clone, Copy)]
struct BhParameters {
    a: f32,
}

struct State{
    bh: BhParameters,

    r: f32, 
    phi: f32,
    theta: f32, 

    p_r: f32,
    p_theta: f32,

    rho: f32, 
    P: f32,
    R: f32,
    Delta: f32,
    Theta: f32,

    b: f32, // Axial angular momentum p_phi
    q: f32, // Carter Constant 

}
let a = Test{a: 1u32, b: 2u32};
impl State {
    // Equations of motion
    fn dr(&self) -> f32 {
        self.Delta/(self.rho*self.rho)*self.p_r
    }
    fn dtheta(&self) -> f32 {
        self.p_theta / (self.rho * self.rho)
    }
    fn dphi(&self) -> f32 {
        // Todo
        -(self.partial_Pb() + self.Delta*self.partial_Thetab())/(2.0*self.Delta*self.rho*self.rho)
    }
    fn dp_r(&self) -> f32 {
        let a = self.bh.a;
        let a2 = a*a;
        let b = self.b;
        let r = self.r;
        let r2 = r*r;
        let cos2theta = self.theta.cos().powi(2);

        -(
            (-self.bh.a.powi(2)+self.r.powi(2)+(self.r-1.0)*self.theta.cos().powi(2))/
            (self.bh.a.powi(2) + self.theta.cos().powi(2)+self.r.powi(2)).powi(2)
        )*self.p_r.powi(2) // https://www.wolframalpha.com/input?i=Partial%5B%28r%5E2-2r%2Ba%5E2%29%2F%5B2%5Br%5E2%2Ba%5E2%2BCos%5B%CE%98%5D%5E2%5D%5D%2Cr%5D
        +(
            self.r/(self.bh.a.powi(2)+self.theta.cos().powi(2) +self.r.powi(2)).powi(2)
        ) * self.p_theta.powi(2) // https://www.wolframalpha.com/input?i=Partial%5B1%2F%5B2%5Br%5E2%2Ba%5E2%2BCos%5B%CE%98%5D%5E2%5D%5D%2Cr%5D
        +(
            (a2 + a*b*r - a*b - r2) / ((a2 + r2 - 2.0*r).powi(2)*(a2 + cos2theta + r2)) - (r*(a2*self.Theta + a2 - a*b + r2*self.Theta + r2 - 2.0*r*self.Theta))/((a2 + r2 - 2.0*r)*(a2 + cos2theta + r2).powi(2))        
        ) //https://www.wolframalpha.com/input?i2d=true&i=Partial%5B%5C%2840%29Divide%5BPower%5Br%2C2%5D%2BPower%5Ba%2C2%5D-ab%2B%5C%2840%29Power%5Br%2C2%5D-2r%2BPower%5Ba%2C2%5D%5C%2841%29T%2C2%5C%2840%29Power%5Br%2C2%5D-2r%2BPower%5Ba%2C2%5D%5C%2841%29%5C%2840%29Power%5Br%2C2%5D%2BPower%5Ba%2C2%5D%2BPower%5BCos%5B%CE%B8%5D%2C2%5D%5C%2841%29%5D%5C%2841%29%2Cr%5D
    }
    fn dp_theta(&self) -> f32 {

        let Q = 2.0*self.Delta * self.rho.powi(2);

        let a = self.bh.a;
        let a2 = a*a;
        let b = self.b;
        let b2 = b*b;
        let r = self.r;
        let r2 = r*r;
        let cos_theta = self.theta.cos();
        let sin_theta = self.theta.sin();
        let cot_theta = cos_theta/sin_theta;

        let sin2_theta = sin_theta*sin_theta;
        let cos2_theta = cos_theta*cos_theta;

        let partial_reciprocal_rho2 = (sin2_theta/(a2 + r2 + cos2_theta).powi(2));


        (
            -0.5*self.Delta*partial_reciprocal_rho2*self.p_r.powi(2)
        )//https://www.wolframalpha.com/input?i2d=true&i=Partial%5BDivide%5B1%2CPower%5Br%2C2%5D%2BPower%5Ba%2C2%5D%2BPower%5BCos%5B%CE%B8%5D%2C2%5D%5D%2C%CE%B8%5D
        +(
            -0.5*partial_reciprocal_rho2*self.p_theta.powi(2)
        )
        +(
            (2.0*self.Delta*(-a2*sin_theta * cos_theta + b2*cot_theta.powi(3) + b2*cot_theta))/Q
        ) // https://www.wolframalpha.com/input?i2d=true&i=Partial%5BDivide%5BR%2B%5C%2840%29T%5C%2841%29%5C%2840%29q-Power%5BCos%5B%CE%B8%5D%2C2%5D%5C%2840%29Divide%5BPower%5Bb%2C2%5D%2CPower%5BSin%5B%CE%B8%5D%2C2%5D%5D-Power%5Ba%2C2%5D%5C%2841%29%5C%2841%29%2CQ%5D%2C%CE%B8%5D
    }
    // fn dp_theta

    // Partial derivatives with respect to b 
    fn partial_Rb(&self) -> f32 {
        2.0*self.P*self.partial_Pb()
    }
    fn partial_Pb(&self) -> f32 {
        -self.bh.a
    }
    fn partial_Thetab(&self) -> f32 {
        (self.q - self.theta.cos().powi(2))*(2.0*self.b)/self.theta.sin().powi(2)
    }

    fn calc_P(r: f32, a: f32, b: f32) -> f32 {
        r*r+a*a-a*b
    }
    fn calc_Delta(r: f32, a: f32) -> f32 {
        r*r-2.0*r+a*a
    }
    fn calc_rho(r: f32, theta: f32, a: f32,) -> f32 {
        (r*r + a*a*theta.cos().powi(2)).powf(0.5)
    }
    fn calc_Theta(theta: f32, a: f32, b: f32, q: f32) -> f32 {
        q - theta.cos().powi(2)*(b*b/theta.sin().powi(2)-a*a)
    }
    fn calc_R(a: f32, b: f32, q:f32, P:f32, Delta: f32) -> f32 {
        P*P - Delta*((b-a).powi(2)+q)
    }
    pub fn new(
        bh: BhParameters, 
        
        r: f32,
        theta:f32, 
        phi:f32, 

        p_r: f32, 
        p_theta: f32,
        
        b:f32,
        q:f32
    ) -> Self {
        let P =  State::calc_P(r, bh.a, b);
        let Delta = State::calc_Delta(r, bh.a);
        Self {
            bh: bh, 

            r: r,
            phi: phi,
            theta: theta,

            p_r: p_r,
            p_theta: p_theta,

            b:b, 
            q:q, 

            rho: State::calc_rho(r, theta, bh.a),
            Delta: Delta,
            Theta:  State::calc_Theta(theta, bh.a, b, q),
            P: P,
            R: State::calc_R(bh.a, b, q, P, Delta)
        }
    }    



    pub fn euler_step(&mut self, h: f32){
        // Calculate derivatives
        let dr = self.dr();
        let dphi = self.dphi();
        let dtheta = self.dtheta();
        let dp_r = self.dp_r();
        let dp_theta = self.dp_theta();

        // Update state values
        self.r += h*dr;
        self.phi += h*dphi;
        self.theta += h*dtheta;
        self.p_r += h*dp_r;
        self.p_theta += h*dp_theta;

        // Update calculated values
        self.rho = State::calc_rho(self.r, self.theta, self.bh.a);
        self.P =  State::calc_P(self.r, self.bh.a, self.b);
        self.Delta = State::calc_Delta(self.r, self.bh.a);
        self.Theta = State::calc_Theta(self.theta, self.bh.a, self.b, self.q);
        self.R = State::calc_R(self.bh.a, self.b, self.q, self.P, self.Delta);

    }

}


struct CartesianObserver { 
    x: f32, 
    y: f32,
    z: f32,
    dx: f32,
    dy: f32,
    dz: f32,
} 
impl Into for CartesianObserver {
    pub fn BoyerLindquist -> BoyerLindquistObserver {

    }
}

struct BoyerLindquistObserver { 
    r: f32,
    theta: f32,
    phi: f32,
    beta: f32, 
    B_r: f32,
    B_theta: f32,
    B_phi: f32,
}


The type of the variable state was redefined, so was lost.


In [120]:
let bh = BhParameters{a: 0.999};

let mut state = State::new(
    bh,
    1.0, 1.0, 1.0,
    1.0, 1.0,  
    1.0, 1.0
);
dbg!((state.r, state.phi, state.theta, state.p_r, state.p_theta));
state.euler_step(0.1);
dbg!((state.r, state.phi, state.theta, state.p_r, state.p_theta));

[src/lib.rs:298] (state.r, state.phi, state.theta, state.p_r, state.p_theta) = (


    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
)
[src/lib.rs:300] (state.r, state.phi, state.theta, state.p_r, state.p_theta) = (
    0.98463356,
    0.72757554,
    1.080876,
    0.8576881,
    1.037065,
)
